In [5]:
%%file HHG_Cal.pyx

import cython
import numpy as np
cimport numpy as np
#cdef extern from "complex.h":
    #pass
#from libc.math import exp

#from libcpp.vector cimport vector
#from libc.math import exp


@cython.boundscheck(False)
@cython.wraparound(False)

cdef inline double complex Dipole(double p):
    return - 1j * 11.3137 * p / (3.1415926 * (p * p + 1) * (p * p + 1) * (p * p + 1))

def HHG_Cal(np.ndarray[dtype = np.double_t, ndim = 1] A, np.ndarray[dtype = np.double_t, ndim =1] E, double dt):

    cdef np.ndarray[dtype = np.double_t, ndim = 1] hhg_temp = np.zeros (A.size)
    cdef int i, j, n
    cdef double A_int, S_int, p_pm
    cdef double complex hhg_int, D_co, D
    n = A.size

    for i in range(n):
    
        A_int = 0.0
        hhg_int = 0.0
        S_int = 0.0
        D_co = 0.0
        D = 0.0
    
        for j in range(1, i):
            A_int = A_int + A[i - j] * dt
            p_pm = - A_int / (j * dt)
            D_co = np.conj (Dipole (p_pm + A[i]))
            D = Dipole (p_pm + A[i - j])
            S_int = S_int + (0.5 * (p_pm + A[i - j]) * (p_pm + A[i - j]) + 0.5) * dt
            hhg_int = hhg_int + (2 * 3.1415926 /(1j * j * dt)) ** 1.5 * D_co * D * E[i - j] * np.exp(-1j * S_int) * dt
        
        hhg_temp[i] = np.real(hhg_int)
        
    return hhg_temp


Overwriting HHG_Cal.pyx


In [6]:
%%file setup.py
from distutils.core import setup
from distutils.extension import Extension
from Cython.Distutils import build_ext
import numpy as np

ext = Extension("HHG_Cal", sources = ["HHG_Cal.pyx"], include_dirs = [np.get_include()])

setup(ext_modules = [ext], cmdclass = {'build_ext': build_ext})

Overwriting setup.py


In [7]:
!python setup.py build_ext --inplace

running build_ext
cythoning HHG_Cal.pyx to HHG_Cal.c
building 'HHG_Cal' extension
gcc -pthread -fno-strict-aliasing -g -O2 -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/home/yizhu/anaconda2/lib/python2.7/site-packages/numpy/core/include -I/home/yizhu/anaconda2/include/python2.7 -c HHG_Cal.c -o build/temp.linux-x86_64-2.7/HHG_Cal.o
In file included from /home/yizhu/anaconda2/lib/python2.7/site-packages/numpy/core/include/numpy/ndarraytypes.h:1788:0,
                 from /home/yizhu/anaconda2/lib/python2.7/site-packages/numpy/core/include/numpy/ndarrayobject.h:18,
                 from /home/yizhu/anaconda2/lib/python2.7/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from HHG_Cal.c:435:
/home/yizhu/anaconda2/lib/python2.7/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:15:2: warning: #warning "Using deprecated NumPy API, disable it by " "#defining NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
 #warning "Using deprecated NumPy A